In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas_profiling

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')

In [ ]:
test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
train['total']=train['RoomService']+train['FoodCourt']+train['ShoppingMall']+train['Spa']+train['VRDeck']
test['total']=test['RoomService']+test['FoodCourt']+test['ShoppingMall']+test['Spa']+test['VRDeck']

In [ ]:
train.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Name','Cabin'],axis=1,inplace=True)
test.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','Name','Cabin'],axis=1,inplace=True)

In [ ]:
def impute_vip (cols):
    VIP = cols[0]
    Age = cols[1]
    total = cols[2]
    if pd.isnull(VIP):
        if Age > 30:
            return True
        elif Age <= 30 and total >1484:
            return True
        else:
            return False

    else:
        return VIP

In [ ]:
train['VIP'] = train[['VIP','Age','total']].apply(impute_vip,axis=1)
test['VIP'] = test[['VIP','Age','total']].apply(impute_vip,axis=1)

In [ ]:
def impute_age (cols):
    Age = cols[0]
    VIP = cols[1]
    if pd.isnull(Age):
        if VIP=='True':
            return 38
        else:
            return 28

    else:
        return Age

In [ ]:
train['Age'] = train[['Age','VIP']].apply(impute_age,axis=1)
test['Age'] = test[['Age','VIP']].apply(impute_age,axis=1)

In [ ]:
def impute_desti (cols):
    Destination = cols[0]
    HomePlanet = cols[1]
    CryoSleep  = cols[2]
    VIP = cols[3]
    if pd.isnull(Destination):
        if HomePlanet=='Mars' and VIP==True:
            return 'TRAPPIST-1e'
        elif HomePlanet=='Earth' and VIP==False:
            return 'TRAPPIST-1e'
        elif HomePlanet=='Europa' and CryoSleep==False:
            return 'TRAPPIST-1e'
    else:
        return Destination

In [ ]:
train['Destination'] = train[['Destination','HomePlanet','CryoSleep','VIP']].apply(impute_desti,axis=1)
test['Destination'] = test[['Destination','HomePlanet','CryoSleep','VIP']].apply(impute_desti,axis=1)

In [ ]:
def impute_cryo (cols):
    CryoSleep = cols[0]
    Destination = cols[1]
    HomePlanet = cols[2]
    if pd.isnull(CryoSleep):
        if Destination=='TRAPPIST-1e' and HomePlanet =='Earth':
            return False
#        else:
#            return False

    else:
        return CryoSleep

In [ ]:
train['CryoSleep'] = train[['CryoSleep','Destination','HomePlanet']].apply(impute_cryo,axis=1)
test['CryoSleep'] = test[['CryoSleep','Destination','HomePlanet']].apply(impute_cryo,axis=1)

In [ ]:
def impute_home (cols):
    HomePlanet = cols[0]
    CryoSleep = cols[1]
    Destination = cols[2]
    if pd.isnull(HomePlanet):
        if CryoSleep==False:
            return 'Earth'
        elif Destination=='55 Cancri e':
            return 'Europa'
    else:
        return HomePlanet

In [ ]:
train['HomePlanet'] = train[['HomePlanet','CryoSleep','Destination']].apply(impute_home,axis=1)
test['HomePlanet'] = train[['HomePlanet','CryoSleep','Destination']].apply(impute_home,axis=1)

In [ ]:
train['total'].fillna(0,inplace=True)
test['total'].fillna(0,inplace=True)

In [ ]:
train['HomePlanet']=train['HomePlanet'].fillna(train['HomePlanet'].mode()[0])
train['CryoSleep']=train['CryoSleep'].fillna(train['CryoSleep'].mode()[0])
train['Destination']=train['Destination'].fillna(train['Destination'].mode()[0])

In [ ]:
test['HomePlanet']=test['HomePlanet'].fillna(test['HomePlanet'].mode()[0])
test['CryoSleep']=test['CryoSleep'].fillna(test['CryoSleep'].mode()[0])
test['Destination']=test['Destination'].fillna(test['Destination'].mode()[0])

In [ ]:
train['CryoSleep']=train['CryoSleep'].replace(False,0)
train['CryoSleep']=train['CryoSleep'].replace(True,1)
train['VIP']=train['VIP'].replace(False,0)
train['VIP']=train['VIP'].replace(True,1)
train['VIP'].astype('int64')
train['Transported']=train['Transported'].replace(False,0)
train['Transported']=train['Transported'].replace(True,1)

In [ ]:
test['CryoSleep']=test['CryoSleep'].replace(False,0)
test['CryoSleep']=test['CryoSleep'].replace(True,1)
test['VIP']=train['VIP'].replace(False,0)
test['VIP']=train['VIP'].replace(True,1)
test['VIP']=train['VIP'].replace('NA','')
test.dropna(inplace=True)
test['VIP'].astype('int64')

In [ ]:
home=pd.get_dummies(train['HomePlanet'],drop_first=True)
destination = pd.get_dummies(train['Destination'],drop_first=True)
home1=pd.get_dummies(test['HomePlanet'],drop_first=True)
destination1 = pd.get_dummies(test['Destination'],drop_first=True)

In [ ]:
train.drop(['HomePlanet','Destination'],axis=1,inplace=True)
test.drop(['HomePlanet','Destination'],axis=1,inplace=True)

In [ ]:
train=pd.concat([train,home,destination],axis=1)
test=pd.concat([test,home1,destination1],axis=1)

In [ ]:
X=train.drop(['Transported'],axis=1)
y=train[['Transported']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc=RandomForestClassifier()

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
pred=rfc.predict(X_test)

In [ ]:
prediction = rfc.predict(test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
ids = test[['PassengerId']].reset_index()
trans = pd.DataFrame({'Transported':prediction})
final = pd.concat([ids,trans],axis=1).drop('index',axis=1)
final['Transported']=final['Transported'].replace(0,False)
final['Transported']=final['Transported'].replace(1,True)
final.to_csv('submission.csv',index=False)